<a href="https://colab.research.google.com/github/Pitap4ik/UkrainianPremierLeague_Analytics/blob/main/UkrainianPremierLeague_DataScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import requests

In [78]:
standings_url = "https://fbref.com/en/comps/39/Ukrainian-Premier-League-Stats"

In [79]:
data = requests.get(standings_url)

In [80]:
from bs4 import BeautifulSoup

In [81]:
soup = BeautifulSoup(data.text)

In [82]:
standings_table = soup.select('table.stats_table')[0]

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if 'squads' in l]

In [ ]:
team_urls = [f"https://Fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data=requests.get(team_url)

In [ ]:
import pandas as pd
from io import StringIO

In [ ]:
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")

In [ ]:
matches[0].head()

In [ ]:
soup=BeautifulSoup(data.text)

In [ ]:
links=soup.find_all('a')

In [ ]:
links=[l.get("href") for l in links]

In [ ]:
links=[l for l in links if l and "all_comps/shooting/" in l]

In [ ]:
shooting_url=f"https://FBref.com{links[0]}"

In [ ]:
data=requests.get(shooting_url)

In [ ]:
shootings =pd.read_html(StringIO(data.text),match = "Shooting")

In [ ]:
shooting = shootings[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on = "Date")

In [ ]:
team_data.head()

In [ ]:
years=list(range(2025,2023,-1))

In [ ]:
years

In [ ]:
standings_url = "https://fbref.com/en/comps/39/Ukrainian-Premier-League-Stats"

In [ ]:
import time
all_matches = []
for year in years:

  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get('href') for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace('-Stats','').replace('-',' ')

    data = requests.get(team_url)
    matches = pd.read_html(StringIO(data.text), match='Scores & Fixtures')[0]

    soup = BeautifulSoup(data.text)
    links = [l.get('href') for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(StringIO(data.text), match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'PK', 'PKatt']], on='Date')
    except ValueError:
      continue

    team_data = team_data[team_data['Comp'] == 'Premier League']
    team_data['Season'] = year
    team_data['Team'] = team_name
    all_matches.append(team_data)
    time.sleep(1)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.to_csv("matches.csv")